In [1]:
import requests
import openai
import torch
from transformers import pipeline
import libs

In [2]:
libs.print_hello()

Hello my capivaras!


In [3]:
#'''
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds with jokes",
    },
    {
        "role": "user",
        "content": "How many helicopters can a human eat in one sitting? Tell me an small story about it",
    },
]
#''';

"""
messages = [
    {
        "role": "system",
        "content": '''You are allowed to call the following function:
1. PluginCapital_get_capital: Returns the name of the capital of a country..
  Arguments: country (Name of the country. Type: string. Required: yes)
2. answer: Send response back to the user. Show all your results here, this is the only thing that the user will see. You won't be able to call any other function after this one. Be sure to return a complete and clear answer, the user will not be able to see any other intermediate messages nor ask for more information. Never mention intermediate messages or results; if you want to mention something, include it here. Call this function only once, with everything you want to show to the user..
  Arguments: text (final textual response to be send to the user. Should use HTML syntax for formatting. Type: string. Required: yes)
</s>''',
    },
    {
        "role": "user",
        "content": "What is the capital of France?",# Answer only with a JSON calling the fucntion PluginCapital_get_capital, nothing else",
        #"content": "What is the capital of France? Answer only with a JSON calling the fucntion PluginCapital_get_capital, nothing else",
        #"content": "Hi",
    },
]
""";

# Test TinyLlama

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...

# Test TinyLlama over LlamaCpp

In [4]:
response = requests.get(url='http://llm-server:8080/health')
response

<Response [200]>

In [ ]:
# Not-streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages},
)
response.json()

In [16]:
# Streaming, chat completion, http client
response = requests.post(
    url='http://llm-server:8080/v1/chat/completions',
    headers={'Content-Type': 'application/json'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
for chunk in response:
    print(chunk[:25], end='')
    #print('---------')

b'data: {"choices":[{"finis'b'THoCMbuyTHBptOZv","model"'b'data: {"choices":[{"finis'b'DYlTHoCMbuyTHBptOZv","mod'b'data: {"choices":[{"finis'b'THoCMbuyTHBptOZv","model"'b'data: {"choices":[{"finis'b'THoCMbuyTHBptOZv","model"'b'data: {"choices":[{"finis'b'mDYlTHoCMbuyTHBptOZv","mo'b'data: {"choices":[{"finis'b'DYlTHoCMbuyTHBptOZv","mod'b'data: {"choices":[{"finis'b'DYlTHoCMbuyTHBptOZv","mod'b'data: {"choices":[{"finis'b'gumDYlTHoCMbuyTHBptOZv","'b'data: {"choices":[{"finis'b'YlTHoCMbuyTHBptOZv","mode'b'data: {"choices":[{"finis'b'DYlTHoCMbuyTHBptOZv","mod'b'data: {"choices":[{"finis'b'YlTHoCMbuyTHBptOZv","mode'b'data: {"choices":[{"finis'b'glttgumDYlTHoCMbuyTHBptOZ'b'data: {"choices":[{"finis'b'umDYlTHoCMbuyTHBptOZv","m'b'data: {"choices":[{"finis'b'tgumDYlTHoCMbuyTHBptOZv",'b'data: {"choices":[{"finis'b'THoCMbuyTHBptOZv","model"'b'data: {"choices":[{"finis'b'tgumDYlTHoCMbuyTHBptOZv",'b'data: {"choices":[{"finis'b'THoCMbuyTHBptOZv","model"'b'data: {"choices":[{"finis'b'lTHoCMbuyTHBptOZv"

In [5]:
client = openai.OpenAI(
    base_url="http://llm-server:8080/v1",
    api_key = "not-needed"
)

In [ ]:
# Not-streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
)
print(type(completion))
print(completion.content)

In [8]:
# Not-streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The answer is: 1 helicopter.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [7]:
#completion.dict()

In [ ]:
# Streaming, text completion, openai client
completion = client.completions.create(
    model="not-needed",
    prompt='The capital of france is',
    max_tokens = 5,
    stream=True,
)
for chunk in completion:
    print(chunk.content, end='')

In [12]:
# Streaming, chat completion, openai client
completion = client.chat.completions.create(
    model="not-needed",
    messages=messages,
    stream=True,
)

for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content or '', end='')
    #print("****************")

A human can eat approximately 10 to 15 helicopters in one sitting. Helicopters are the popular snacks in certain cultures.

# Test TinyLlama over LlamaCpp over proxy

In [4]:
requests.get('http://orchestrator-proxy:80/docs')

<Response [200]>

In [11]:
import json

In [19]:
response = requests.post(
    url='http://orchestrator-proxy:80/v1/chat/completions',
    stream=True,
)
buffer = ""
for chunk in response:
    if chunk:
        buffer += chunk.decode('utf-8')
        #print('buffer is now:', buffer)
        try:
            # Try to load the JSON object
            json_obj = json.loads(buffer)
            print(json_obj)
            buffer = ""
        except json.JSONDecodeError:
            # If JSONDecodeError is raised, wait for more data
            continue

{'answer': 'I have placed an order for ', 'citations': None, 'visualizations': None, 'intermediate_results': None}
{'answer': '100 units of the "Untersuchungshandschuhe Nitril light lemon Gr. M" glove model', 'citations': None, 'visualizations': None, 'intermediate_results': None}
{'answer': ', as requested. Additionally, I have closed the ticket regarding the defective product.', 'citations': None, 'visualizations': None, 'intermediate_results': None}
{'answer': ' Is there anything else I can assist you with?', 'citations': None, 'visualizations': None, 'intermediate_results': None}


In [14]:
buffer

'{"answer":"I have placed an order for ","citations":null,"visualizations":null,"intermediate_results":null}{"answer":"100 units of the \\"Untersuchungshandschuhe Nitril light lemon Gr. M\\" glove model","citations":null,"visualizations":null,"intermediate_results":null}{"answer":", as requested. Additionally, I have closed the ticket regarding the defective product.","citations":null,"visualizations":null,"intermediate_results":null}{"answer":" Is there anything else I can assist you with?","citations":null,"visualizations":null,"intermediate_results":null}'

In [16]:
json.loads('''{"answer":"I have placed an order for ","citations":null,"visualizations":null,"intermediate_results":null}''')

{'answer': 'I have placed an order for ',
 'citations': None,
 'visualizations': None,
 'intermediate_results': None}

In [9]:
# Streaming, chat completion, http client
# Works, but return all at once
response = requests.post(
    url='http://orchestrator-proxy:80/v1/chat/completions',
    headers={'Content-Type': 'application/json', 'Authorization': 'not-needed'},
    json={'messages': messages, 'stream': True},
    stream=True,
)
chunks = []
for chunk in response:
    print(chunk[:25], end='')
    print('\n---------')
    chunks.append(chunk)

b'{"answer":"I have placed '
---------
b'{"answer":"100 units of t'
---------
b'null,"intermediate_result'
---------
b'{"answer":", as requested'
---------
b'ions":null,"intermediate_'
---------
b'{"answer":" Is there anyt'
---------


In [13]:
buffer = ""
for chunk in response.iter_lines():
    if chunk:
        buffer += chunk.decode('utf-8')
        try:
            # Try to load the JSON object
            json_obj = json.loads(buffer)
            print(json_obj)
            buffer = ""
        except json.JSONDecodeError:
            # If JSONDecodeError is raised, wait for more data
            continue

[b'{"answer":"I have placed an order for ","citations":null,"visualizations":null,"intermediate_results":null}',
 b'{"answer":"100 units of the \\"Untersuchungshandschuhe Nitril light lemon Gr. M\\" glove model","citations":null,"visualizations":',
 b'null,"intermediate_results":null}',
 b'{"answer":", as requested. Additionally, I have closed the ticket regarding the defective product.","citations":null,"visualizat',
 b'ions":null,"intermediate_results":null}',
 b'{"answer":" Is there anything else I can assist you with?","citations":null,"visualizations":null,"intermediate_results":null}']